# 第六节 研究Agent智能体及其应用实践

### 课程笔记大纲

1. LLM作为推理引擎的概念
2. LangChain中的Agents
3. 初始化Agents
4. 使用Agents
5. Agents的实际应用示例

### 环境安装配置

首先，导入所需的环境变量和依赖包，需要确保机器上安装了相应的Python包。开发者可以通过以下命令轻松完成安装：

In [ ]:
%pip install --upgrade --quiet  openai langchain-openai langchain langchainhub

导入OpenAI模块,设置模型型号为`"gpt-3.5-turbo"`，设置好开发密钥。

In [ ]:
import os
API_SECRET_KEY = "填写【个人资料】中获取的最新token"
BASE_URL = "https://www.ai360labs.com/openai/v1/"

os.environ["OPENAI_API_KEY"] = API_SECRET_KEY
os.environ["OPENAI_BASE_URL"] = BASE_URL

## LLM作为Agent的推理引擎

LLM的角色和作用：
   - 大型语言模型（LLM）扮演着类似于知识存储库的角色，其目的在于理解和处理大量的自然语言文本数据。
   - LLM可以被视为一个巨大的语言学习器，通过学习大量的文本数据，它能够捕捉到语言的模式、语义和逻辑。
   - 在应用中，LLM可以用于各种任务，包括文本生成、文本理解、情感分析、问答系统等。

## DIY 智能体的工具箱

当ChatGPT于2023年3月份横空出世时，人们惊讶地发现它拥有广泛的知识和能力，然而，令人遗憾的是，它的计算能力很差，数学能力不及中国的小学生（GPT-4等高级模型有所改观）。

为了解决这个问题，我们可以动手DIY，为我们的智能体增加处理数学问题的能力。我们可以采取以下步骤：

定义三个工具函数：加法、乘法、指数运算。

In [ ]:
from langchain_core.tools import tool


@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [add,multiply,exponentiate]

导入需要使用的模块。

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

从现有hub中拉取`openai-tools-agent`提示词模板。

In [ ]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

使用create_openai_tools_agent方法创建一个智能体。

In [ ]:
# Choose the LLM that will drive the agent
# Only certain models support this
model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(model, tools, prompt)

创建一个Agent执行器，将工具和agent实例传递给执行器环境。

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

唤起智能体，查看智能体如何选择工具，回答不同类型的问题。

In [ ]:
agent_executor.invoke(
    {
        "input": "将3的五次方与十二和三的和相乘，然后对整个结果进行平方。"
    }
)

**注意**:

智能体的回答有时候是错误的，或者网络问题导致错误，如果碰到这样的情况，请重新唤醒一下智能体。

如果执行从hub拉取提示词模板的代码出现错误，请重新启动该笔记本，如果你本地环境有多个python版本，请尝试更换python的版本。这种错误是由于python版本所致，另外还可以升级本地的python版本解决。